In [3]:
!pip install weaviate-client==4.7.0rc0

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for weaviate-client==4.7.0rc0 from https://files.pythonhosted.org/packages/3a/22/139efcb8c311d7e05766e926107cfb3ec0530b6242cfdb8a387961b46968/weaviate_client-4.7.0rc0-py3-none-any.whl.metadata
  Using cached weaviate_client-4.7.0rc0-py3-none-any.whl.metadata (3.3 kB)
Using cached weaviate_client-4.7.0rc0-py3-none-any.whl (365 kB)
  Attempting uninstall: weaviate-client
    Found existing installation: weaviate-client 4.6.3
    Uninstalling weaviate-client-4.6.3:
      Successfully uninstalled weaviate-client-4.6.3


In [4]:
import weaviate
import weaviate.classes as wvc
import requests
import json

In [5]:
client = weaviate.connect_to_wcs(
    cluster_url="",
    auth_credentials=weaviate.auth.AuthApiKey("")
    
)

In [7]:

#await client.is_ready()
client.is_ready()

True

In [8]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType
client.collections.create(
    name="NameSearch",
    properties=[
        wc.Property(name="PersonName", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(),
)

In [9]:
def VectorElement(word):
    ListAlphabets = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    ListVectors=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    count = 0
    while count < 26:
        ListVectors[count]= sum(1 for element in word if element.lower()==ListAlphabets[count].lower())
        count =count+1
    return ListVectors

In [11]:
from faker import Faker  
import time

t0 = time.time()

fake = Faker()  
DocumentSearch = client.collections.get("NameSearch")
with DocumentSearch.batch.dynamic() as batch:
    for i in range(0, 100000): 
        fakename =fake.name()
        if (i==1):
            print(fakename)  
        properties = {"personName": fakename}
        vector=VectorElement(fakename)
        #await batch.add_object(properties,vector=vector)
        batch.add_object(properties,vector=vector)
t1 = time.time()
total = t1-t0
print(total)  

John Garza
67.610600233078


In [12]:
import weaviate.classes.query as wq
query_text = "John Garza"
import time
query_vector= VectorElement(query_text)
# Get the collection
SkillSet= client.collections.get("NameSearch")


t0 = time.time()
# Perform query
response = SkillSet.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["personName"] 
    )  
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P
t1 = time.time()
total = t1-t0
print('Time Taken')
print(total)  

John Garza
Distance to query: 0.000

John Garza
Distance to query: 0.000

John Garza
Distance to query: 0.000

Joan Garza
Distance to query: 0.066

Jonathan Garza
Distance to query: 0.072

Time Taken
0.6111805438995361
